In [1]:
!pip install transformers accelerate bitsandbytes>0.37.0
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 7.4 MB/s eta 0:00:00


In [2]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
from huggingface_hub import login
login(token=user_secrets.get_secret("HF_READ"))
login(token=user_secrets.get_secret("HF_WRITE"))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


# Import Libraries

In [3]:
from datasets import load_dataset
datasets = load_dataset("Salesforce/xlam-function-calling-60k")

Generating train split:   0%|          | 0/60000 [00:00<?, ? examples/s]

In [4]:
raw_dataset = datasets['train'].train_test_split(test_size=0.1,seed=1)

In [5]:
raw_dataset['train'] = raw_dataset['train'].select([*range(2000)])
raw_dataset['test'] = raw_dataset['train'].select([*range(100)])

# Retrieve Model

In [6]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer,AutoModelForCausalLM
from peft import PeftModel, PeftConfig, get_peft_model
import torch

In [7]:
base_model = AutoModelForCausalLM.from_pretrained("SwastikM/Meta-Llama-3-8B-Instruct_bitsandbytes_4bit",device_map="auto")
model = PeftModel.from_pretrained(base_model, "SwastikM/Meta-Llama3-8B-Chat-Adapter")
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


model.safetensors.index.json:   0%|          | 0.00/132k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.65G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

...

adapter_model.safetensors:   0%|          | 0.00/13.6M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
model = model.to("cuda")
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=Fal

In [9]:
x1 = {"role": "system", "content": """You are a APIGen Function Calling Tool. You will br provided with a user query and associated tools for answering the query.
    query (string): The query or problem statement.
    tools (array): An array of available tools that can be used to solve the query.
    Each tool is represented as an object with the following properties:
        name (string): The name of the tool.
        description (string): A brief description of what the tool does.
        parameters (object): An object representing the parameters required by the tool.
            Each parameter is represented as a key-value pair, where the key is the parameter name and the value is an object with the following properties:
                type (string): The data type of the parameter (e.g., "int", "float", "list").
                description (string): A brief description of the parameter.
                required (boolean): Indicates whether the parameter is required or optional.
    You will provide the Answer array.
        Answers array provides the specific tool and arguments used to generate each answer."""}
x2 = {"role": "user", "content": None}
x3 = {"role": "assistant", "content": None}
user_template = 'Query: {Q} Tools: {T}'
response_template = '{A}'
Q = "Where can I find live giveaways for beta access and games?"
T = """[{"name": "live_giveaways_by_type", "description": "Retrieve live giveaways from the GamerPower API based on the specified type.", "parameters": {"type": {"description": "The type of giveaways to retrieve (e.g., game, loot, beta).", "type": "str", "default": "game"}}}]"""

In [10]:
x2['content'] = f'{user_template.format(Q=Q,T=T)}'
prompts = [x1,x2]
input_ids = tokenizer.apply_chat_template(
    prompts,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [11]:
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
2024-07-21 19:18:33.906664: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 19:18:33.906791: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 19:18:34.036417: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable

In [12]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

[{"name": "live_giveaways_by_type", "arguments": {"type": "beta"}}, {"name": "live_giveaways_by_type", "arguments": {"type": "game"}}]


In [13]:
raw_dataset['test']

Dataset({
    features: ['query', 'id', 'answers', 'tools'],
    num_rows: 100
})

In [14]:
for item in raw_dataset['test']:
    Q = item['query']
    T = item['tools']
    A = item['answers']
    x2['content'] = f'{user_template.format(Q=Q,T=T)}'
    x3['content'] = f'{response_template.format(A=A)}'
    prompts = [x1,x2]
    input_ids = tokenizer.apply_chat_template(
        prompts,
        add_generation_prompt=True,
        return_tensors="pt"
        ).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            max_new_tokens=256,
            eos_token_id=terminators
            )
    
    response = outputs[0][input_ids.shape[-1]:]
    print(f"Query : {Q} \n")
#     print(f"Tools : {T} \n")
    print(f"Answer : {A} \n")
    print(f"Output : {tokenizer.decode(response, skip_special_tokens=True)}")
    print("==========================================================================")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Query : Get a random Counter-Strike skin from the 'Sniper' class and search for E-Sports players, teams, and tournaments related to 'Counter-Strike: Global Offensive'. 

Answer : [{"name": "random_skin_from_class", "arguments": {"class_name": "Sniper"}}, {"name": "search", "arguments": {"term": "Counter-Strike: Global Offensive"}}] 

Output : [{"name": "random_skin_from_class", "arguments": {"class_name": "Sniper"}}, {"name": "search", "arguments": {"term": "Counter-Strike: Global Offensive"}}]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Query : Search for news on the financial success of recent blockbuster movies with a limit of 7 and an offset of 30. 

Answer : [{"name": "basic_search", "arguments": {"q": "blockbuster movies financial success", "limit": 7, "offset": 30}}] 

Output : [{"name": "basic_search", "arguments": {"q": "financial success recent blockbuster movies", "limit": 7, "offset": 30}}]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


...

Query : Get the catalog albums chart for the week starting October 1, 2022. 

Answer : [{"name": "catalog_albums", "arguments": {"week": "2022-10-01"}}] 

Output : [{"name": "catalog_albums", "arguments": {"week": "2022-10-01"}}]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Query : Get information about stickers with ids 123, 456, and 789 using the RapidAPI service. 

Answer : [{"name": "v1_sticker", "arguments": {"sticker_ids": "123,456,789"}}] 

Output : [{"name": "v1_sticker", "arguments": {"sticker_ids": "123,456,789"}}]


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Query : What is the information about the TV show with ID 12345? 

Answer : [{"name": "shows_id", "arguments": {"is_id": 12345}}] 

Output : [{"name": "shows_id", "arguments": {"is_id": 12345}}]
Query : Check the validity of parentheses in '(({[]}))' and count the occurrences of each color in ['red', 'blue', 'red', 'green', 'blue', 'blue', 'red']. 

Answer : [{"name": "is_valid_parentheses", "arguments": {"s": "(({[]}))"}}, {"name": "count_occurrences", "arguments": {"lst": ["red", "blue", "red", "green", "blue", "blue", "red"]}}] 

Output : [{"name": "is_valid_parentheses", "arguments": {"s": "(({[]}))"}}, {"name": "count_occurrences", "arguments": {"lst": ["red", "blue", "red", "green", "blue", "blue", "red"]}}]
